In [1]:
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

## Getting the Data

In [2]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip

--2021-06-10 20:38:45--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.20.128, 108.177.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   122MB/s    in 1.2s    

2021-06-10 20:38:46 (122 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [3]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip 

--2021-06-10 20:38:46--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.135.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘validation-horse-or-human.zip’

validation-horse-or 100%[===================>]  10.95M  26.4MB/s    in 0.4s    

2021-06-10 20:38:47 (26.4 MB/s) - ‘validation-horse-or-human.zip’ saved [11480187/11480187]



### Unziping training data

In [4]:
!unzip "horse-or-human.zip" -d "training/"

Archive:  horse-or-human.zip
  inflating: training/horses/horse01-0.png  
  inflating: training/horses/horse01-1.png  
  inflating: training/horses/horse01-2.png  
  inflating: training/horses/horse01-3.png  
  inflating: training/horses/horse01-4.png  
  inflating: training/horses/horse01-5.png  
  inflating: training/horses/horse01-6.png  
  inflating: training/horses/horse01-7.png  
  inflating: training/horses/horse01-8.png  
  inflating: training/horses/horse01-9.png  
  inflating: training/horses/horse02-0.png  
  inflating: training/horses/horse02-1.png  
  inflating: training/horses/horse02-2.png  
  inflating: training/horses/horse02-3.png  
  inflating: training/horses/horse02-4.png  
  inflating: training/horses/horse02-5.png  
  inflating: training/horses/horse02-6.png  
  inflating: training/horses/horse02-7.png  
  inflating: training/horses/horse02-8.png  
  inflating: training/horses/horse02-9.png  
  inflating: training/horses/horse03-0.png  
  inflating: training/hors

### Unziping validation data

In [5]:
!unzip "validation-horse-or-human.zip" -d "validation/"

Archive:  validation-horse-or-human.zip
  inflating: validation/horses/horse1-000.png  
  inflating: validation/horses/horse1-105.png  
  inflating: validation/horses/horse1-122.png  
  inflating: validation/horses/horse1-127.png  
  inflating: validation/horses/horse1-170.png  
  inflating: validation/horses/horse1-204.png  
  inflating: validation/horses/horse1-224.png  
  inflating: validation/horses/horse1-241.png  
  inflating: validation/horses/horse1-264.png  
  inflating: validation/horses/horse1-276.png  
  inflating: validation/horses/horse1-298.png  
  inflating: validation/horses/horse1-335.png  
  inflating: validation/horses/horse1-384.png  
  inflating: validation/horses/horse1-411.png  
  inflating: validation/horses/horse1-436.png  
  inflating: validation/horses/horse1-455.png  
  inflating: validation/horses/horse1-484.png  
  inflating: validation/horses/horse1-510.png  
  inflating: validation/horses/horse1-539.png  
  inflating: validation/horses/horse1-554.png  


## Data Preprocessing

### Training data

In [6]:
train_datagen = ImageDataGenerator(rescale=1/255.0)

train_datagenrator = train_datagen.flow_from_directory(
    "training/",
    target_size = (300,300),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


### Validation data

In [7]:
valid_datagen = ImageDataGenerator(rescale = 1/255.0)

valid_datagenrator = valid_datagen.flow_from_directory(
    "validation/",
    target_size = (300,300),
    batch_size = 32,
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


## Defining and compiling a model

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [9]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

## Training the Neural Network

In [10]:
model.fit(train_datagenrator,
          steps_per_epoch = 8,
          epochs = 20,
          validation_data = valid_datagenrator,
          validation_steps = 8,
          verbose = 1)

Epoch 1/20
8/8 [==============================] - 85s 10s/step - loss: 0.8292 - accuracy: 0.5050 - val_loss: 0.6232 - val_accuracy: 0.7656
Epoch 2/20
8/8 [==============================] - 82s 10s/step - loss: 0.6606 - accuracy: 0.6963 - val_loss: 0.5961 - val_accuracy: 0.7227
Epoch 3/20
8/8 [==============================] - 90s 11s/step - loss: 2.6153 - accuracy: 0.7852 - val_loss: 0.5060 - val_accuracy: 0.8125
Epoch 4/20
8/8 [==============================] - 85s 11s/step - loss: 0.2787 - accuracy: 0.8888 - val_loss: 0.5460 - val_accuracy: 0.8828
Epoch 5/20
8/8 [==============================] - 82s 10s/step - loss: 0.2566 - accuracy: 0.8910 - val_loss: 0.8292 - val_accuracy: 0.8438
Epoch 6/20
8/8 [==============================] - 81s 10s/step - loss: 0.1696 - accuracy: 0.9344 - val_loss: 1.2425 - val_accuracy: 0.8047
Epoch 7/20
8/8 [==============================] - 81s 10s/step - loss: 0.1422 - accuracy: 0.9466 - val_loss: 0.5922 - val_accuracy: 0.8945
Epoch 8/20
8/8 [===========